---
title: Downloading Population Data from from WorldPop
short_title: Population Data
---

In this notebook we will use `dhis2eo` to download [population count data from WorldPop](https://hub.worldpop.org/geodata/listing?id=135) for a set of DHIS2 organisation units. 

----------------------------------------
## Requirements

### 1. Connect to DHIS2

In order to run this notebook, you first need to connect to an instance of DHIS2. For our example, we will connect to a local instance of DHIS2 containing the standard Sierra Leone demo database, but you should be able to switch out the instance url and credentials to work directly with your own database. 

In [2]:
from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

# Create DHIS2 client connection
cfg = ClientSettings(
  base_url="https://play.im.dhis2.org/stable-2-42-3",#"http://localhost:8080",
  username="admin",
  password="district"
)
client = DHIS2Client(settings=cfg)

# Verify connection
info = client.get_system_info()
print("Current DHIS2 version:", info["version"])

Current DHIS2 version: 2.42.3


-------------
## Step-by-Step Workflow

Start by importing the libraries that we need:

In [3]:
import dhis2eo
import dhis2eo.org_units
import dhis2eo.data.worldpop

### Step 1: Retrieve organisation units

First we get the organisation units from DHIS2 and load them into a format we can work with: 

In [4]:
org_units_geojson = client.get_org_units_geojson(level=2)
org_units = dhis2eo.org_units.from_dhis2_geojson(org_units_geojson)
org_units

,org_unit_id,name,level,geometry
0,O6uvpzGd5pu,Bo,2,"POLYGON ((-11.5914 8.4875, -11.5906 8.4769, -1..."
1,fdc6uOvgoji,Bombali,2,"POLYGON ((-11.8091 9.2032, -11.8102 9.1944, -1..."
2,lc3eMKXaEfw,Bonthe,2,"MULTIPOLYGON (((-12.5568 7.3832, -12.5574 7.38..."
3,jUb8gELQApl,Kailahun,2,"POLYGON ((-10.7972 7.5866, -10.8002 7.5878, -1..."
4,PMa2VCrupOd,Kambia,2,"MULTIPOLYGON (((-13.1349 8.8471, -13.1343 8.84..."
5,kJq2mPyFEHo,Kenema,2,"POLYGON ((-11.3596 8.5317, -11.3513 8.5234, -1..."
6,qhqAxPSTUXp,Koinadugu,2,"POLYGON ((-10.585 9.0434, -10.5877 9.0432, -10..."
7,Vth0fbpFcsO,Kono,2,"POLYGON ((-10.585 9.0434, -10.5848 9.0432, -10..."
8,jmIPBj66vD6,Moyamba,2,"MULTIPOLYGON (((-12.6351 7.6613, -12.6346 7.66..."
9,TEQlaapDQoK,Port Loko,2,"MULTIPOLYGON (((-13.119 8.4718, -13.1174 8.470..."


### Step 2: Download data for a single year

We provide a convenience function for downloading WorldPop historical and projected population data from the [WorldPop v2 dataset between 2015 and 2030](https://hub.worldpop.org/project/categories?id=3). Let's get the population in Sierra Leone projected for 2030:

In [5]:
year = 2030
country_code = 'SLE'
data = dhis2eo.data.worldpop.get_population_data(year, country_code)
print(f'Population data {data}')

dhis2eo.data.worldpop - INFO - Loading from cache: C:\Users\karimba\AppData\Local\Temp\sle_pop_2030_CN_100m_R2025A_v1.tif
Population data <xarray.Dataset> Size: 54MB
Dimensions:    (x: 3635, y: 3695)
Coordinates:
  * x          (x) float64 29kB -13.3 -13.3 -13.3 -13.3 ... -10.27 -10.27 -10.27
  * y          (y) float64 30kB 10.0 9.999 9.998 9.997 ... 6.923 6.922 6.921
    year       int64 8B 2030
Data variables:
    total_pop  (y, x) float32 54MB ...
Attributes:
    TIFFTAG_DOCUMENTNAME:      sle_pop_2030_CN_100m_R2025A_v1
    TIFFTAG_IMAGEDESCRIPTION:  SLE population 2030 [WorldPop R2025A v1]
    TIFFTAG_DATETIME:          2025-07-29 04:37:30
    TIFFTAG_COPYRIGHT:         CC-BY-4.0
    Description:               SLE population 2030 [WorldPop R2025A v1]
    AREA_OR_POINT:             Area


### Step 3: Download data for multiple years

Downloading data for a range of years can be achieved with a simple loop: 

In [7]:
for year in range(2015, 2025+1):
    print(f'Year: {year}')

    # Download data...
    data = dhis2eo.data.worldpop.get_population_data(year, country_code)

    # do something with the data
    # e.g. save to disk, aggregate, or import to DHIS2
    # ...

Year: 2015
dhis2eo.data.worldpop - INFO - Loading from cache: /tmp/sle_pop_2015_CN_100m_R2025A_v1.tif
Year: 2016
dhis2eo.data.worldpop - INFO - Loading from cache: /tmp/sle_pop_2016_CN_100m_R2025A_v1.tif
Year: 2017
dhis2eo.data.worldpop - INFO - Loading from cache: /tmp/sle_pop_2017_CN_100m_R2025A_v1.tif
Year: 2018
dhis2eo.data.worldpop - INFO - Loading from cache: /tmp/sle_pop_2018_CN_100m_R2025A_v1.tif
Year: 2019
dhis2eo.data.worldpop - INFO - Loading from cache: /tmp/sle_pop_2019_CN_100m_R2025A_v1.tif
Year: 2020
dhis2eo.data.worldpop - INFO - Loading from cache: /tmp/sle_pop_2020_CN_100m_R2025A_v1.tif
Year: 2021
dhis2eo.data.worldpop - INFO - Loading from cache: /tmp/sle_pop_2021_CN_100m_R2025A_v1.tif
Year: 2022
dhis2eo.data.worldpop - INFO - Loading from cache: /tmp/sle_pop_2022_CN_100m_R2025A_v1.tif
Year: 2023
dhis2eo.data.worldpop - INFO - Loading from cache: /tmp/sle_pop_2023_CN_100m_R2025A_v1.tif
Year: 2024
dhis2eo.data.worldpop - INFO - Loading from cache: /tmp/sle_pop_2024_CN

## Next steps

In this notebook we only showed how to download the WorldPop data. Possible next steps include: 

* [Aggregate the data to your organisation units](../../aggregation/org-unit-aggregation.ipynb).
* [Import the aggregated data to DHIS2](../../import-data/using-python-client.ipynb).
* Automatically trigger this script at regular intervals, e.g. via a `cron` job (guide to be added in the future). 